In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load raw data
df = pd.read_csv(r"C:/Users/BALA/OneDrive - University of Hertfordshire/Desktop/mutual-fund-recommender/data/raw/Mutual_Funds.csv")
print("Loaded:", df.shape)
df.head()


Loaded: (29033646, 7)


,Fund_House,Scheme_Type,Scheme_Category,Scheme_Code,Scheme_Name,Date,NAV
0,Standard Chartered Mutual Fund,Open Ended Schemes,Income,100027,Grindlays Super Saver Income Fund-GSSIF-Half Y...,29-05-2008,10.7205
1,Standard Chartered Mutual Fund,Open Ended Schemes,Income,100027,Grindlays Super Saver Income Fund-GSSIF-Half Y...,28-05-2008,10.7250
2,Standard Chartered Mutual Fund,Open Ended Schemes,Income,100027,Grindlays Super Saver Income Fund-GSSIF-Half Y...,27-05-2008,10.7216
3,Standard Chartered Mutual Fund,Open Ended Schemes,Income,100027,Grindlays Super Saver Income Fund-GSSIF-Half Y...,26-05-2008,10.7206
4,Standard Chartered Mutual Fund,Open Ended Schemes,Income,100027,Grindlays Super Saver Income Fund-GSSIF-Half Y...,23-05-2008,10.7152


In [3]:
# Handle Dates & Sort
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y', errors='coerce')
df = df.dropna(subset=['Date'])
df = df.sort_values(['Scheme_Code', 'Date'])


In [4]:
# Clean NAV Column

df['NAV'] = pd.to_numeric(df['NAV'], errors='coerce')
df = df.dropna(subset=['NAV'])


In [5]:
# Filter to Schemes with Enough Data 
# Only using schemes with >100 data points
scheme_counts = df['Scheme_Code'].value_counts()
valid_schemes = scheme_counts[scheme_counts > 100].index.tolist()
df = df[df['Scheme_Code'].isin(valid_schemes)]
print("Schemes with enough data:", len(valid_schemes))

print("Number of valid schemes:", len(valid_schemes))



Schemes with enough data: 28936
Number of valid schemes: 28936


In [9]:
from sklearn.preprocessing import MinMaxScaler

# Define a function to scale NAV per group
def scale_group(group):
    scheme_code = group['Scheme_Code'].iloc[0]  # grab group label manually
    scaler = MinMaxScaler()
    group['NAV_scaled'] = scaler.fit_transform(group[['NAV']])
    scaler_dict[scheme_code] = scaler
    return group

scaler_dict = {}

# Group and apply without include_group
df_scaled = (
    df[df['Scheme_Code'].isin(valid_schemes)]
    .groupby('Scheme_Code', group_keys=False)
    .apply(scale_group)
)

df_scaled.reset_index(drop=True, inplace=True)


C:\Users\BALA\AppData\Local\Temp\ipykernel_1924\1053152452.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(scale_group)


In [10]:
#Save the Preprocessed CSV
df_scaled.to_csv("C:/Users/BALA/OneDrive - University of Hertfordshire/Desktop/mutual-fund-recommender/data/processed/preprocessed_mutual_funds.csv", index=False)
print("Preprocessed data saved to data/processed/preprocessed_mutual_funds.csv")


Preprocessed data saved to data/processed/preprocessed_mutual_funds.csv
